# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [5]:
#include <stdio.h>

int main() {
    double S = 200, J = 1;
    int N = 5;
    for (int i = 0; i < N; i++) {
        printf("%lf\n", S);
        S *= 1 + J/100;
    }
    return 0;
}

200.000000
202.000000
204.020000
206.060200
208.120802


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [7]:
#include <stdio.h>

double proximaParcela(double S, double J) {
    return S * (1 + J/100);
}

int main() {
    double S = 200, J = 1;
    int N = 5;
    for (int i = 0; i < N; i++) {
        printf("%lf\n", S);
        S = proximaParcela(S, J);
    }
    return 0;
}

200.000000
202.000000
204.020000
206.060200
208.120802


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [8]:
#include <stdio.h>

double proximaParcela(double S, double J) {
    return S * (1 + J/100);
}

void calcular_parcelas(double S, double J, int N) {
    for (int i = 0; i < N; i++) {
        printf("%lf\n", S);
        S = proximaParcela(S, J);
    }
}

int main() {
    double S = 200, J = 1;
    int N = 5;
    calcular_parcelas(S, J, N);
    return 0;
}

200.000000
202.000000
204.020000
206.060200
208.120802


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [1]:
#include <stdio.h>

typedef struct {
    double S;
    int N;
    double J;
    int pagos;
} Emprestimo;

void proximaParcela(Emprestimo *e) {
    if (e->pagos < e->N - 1) {
        (e->pagos)++;
        e->S *= (1 + e->J/100);
    } else {
        e->S = 0;
    }
}

void informar_parcelas(Emprestimo *e) {
    while (e->S > 0) {
        printf("%lf\n", e->S);
        proximaParcela(e);
    }
}

int main() {
    Emprestimo e;
    e.S = 200;
    e.N = 5;
    e.J = 1;
    e.pagos = 0;
    informar_parcelas(&e);
}

200.000000
202.000000
204.020000
206.060200
208.120802


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [5]:
#include <stdio.h>
#include <stdlib.h>
#define Q 2

typedef struct {
    int idx;
    double S;
    int N;
    double J;
    int pagos;
} Emprestimo;

typedef struct no_lista_ligada_circ_emprestimos {
    struct no_lista_ligada_circ_emprestimos *ant;
    Emprestimo e;
    struct no_lista_ligada_circ_emprestimos *prox;
} no_emp;

void proximaParcela(Emprestimo *e) {
    if (e->pagos < e->N - 1) {
        (e->pagos)++;
        e->S *= (1 + e->J/100);
    } else {
        e->S = 0;
    }
}

void informar_parcelas(no_emp *lista) {
    no_emp *prox = lista;
    Emprestimo *e;
    while (prox != NULL) {
        lista = prox;
        e = &(lista->e);
        printf("Emprestimo %d: parcela %d eh %lf\n", e->idx, e->pagos + 1, e->S);
        proximaParcela(e);
        if (e->S > 0) {
            prox = lista->prox;
        } else {
            if (lista->prox == lista){
                prox = NULL;
            } else {
                prox = lista->prox;
                prox->ant = lista->ant;
                lista->ant->prox = prox;
            }
            free(lista);
        }
    }
}

int main() {
    no_emp *lista, *aux = NULL;
    Emprestimo *emp;
    for (int i = 0; i < Q; i++) {
        lista = malloc(sizeof(no_emp));
        emp = &(lista->e);
        emp->idx = i + 1;
        emp->S = 200 + 300 * i;
        emp->N = 5 + 2 * i;
        emp->J = 1 + i;
        emp->pagos = 0;
        lista->ant = aux;
        if (aux != NULL) {
            aux->prox = lista;
        }
        aux = lista;
    }
    while (aux->ant != NULL) {
        aux = aux->ant;
    }
    lista->prox = aux;
    aux->ant = lista;
    lista = aux;
    informar_parcelas(lista);
}

Emprestimo 1: parcela 1 eh 200.000000
Emprestimo 2: parcela 1 eh 500.000000
Emprestimo 1: parcela 2 eh 202.000000
Emprestimo 2: parcela 2 eh 510.000000
Emprestimo 1: parcela 3 eh 204.020000
Emprestimo 2: parcela 3 eh 520.200000
Emprestimo 1: parcela 4 eh 206.060200
Emprestimo 2: parcela 4 eh 530.604000
Emprestimo 1: parcela 5 eh 208.120802
Emprestimo 2: parcela 5 eh 541.216080
Emprestimo 2: parcela 6 eh 552.040402
Emprestimo 2: parcela 7 eh 563.081210
